In [1]:
from data_utils import load_data, flatten_data
from early_model import stacked_lstm
from model_utils import model_pipeline, plot_history
import numpy as np
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
from scipy.stats import ttest_ind
from scipy.stats import spearmanr
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import BatchNormalization, Activation

In [2]:
train_participant_num = ["C56D", "C93D", "C382D", "C382N", "C544D", "C709N", "C788N", "P113D", "P113N", "P191D", "P191N", "P299D", "P299N", "P300D", "P336D", "P492D", "P492N", "P531N", "P699D", "P699N", "P890N", "P921D", "P921N"]
valid_participant_num = ["C67D", "C202D", "C202N", "C256D", "C256N", "P54D", "P54N", "P342D", "P342N", "P487D", "P487N", "P649N"]

X_train, y_train = load_data(train_participant_num, 'train', downsampling=True, angle_energy=False, augment=False)
X_valid, y_valid = load_data(valid_participant_num, 'validation')

num_classes = y_train.shape[1]

C56D is not selected to be used for training (downsampling)
C93D is not selected to be used for training (downsampling)
C382D is not selected to be used for training (downsampling)
C382N is not selected to be used for training (downsampling)
C544D is not selected to be used for training (downsampling)
C709N is not selected to be used for training (downsampling)
C788N is not selected to be used for training (downsampling)
P113D is selected to be used for training (downsampling)
P113N is selected to be used for training (downsampling)
P191D is selected to be used for training (downsampling)
P191N is selected to be used for training (downsampling)
P299D is selected to be used for training (downsampling)
P299N is selected to be used for training (downsampling)
P300D is selected to be used for training (downsampling)
P336D is selected to be used for training (downsampling)
P492D is selected to be used for training (downsampling)
P492N is selected to be used for training (downsampling)
P531N

In [3]:
#X,Y,Z Coordinates
X_train_pose = X_train[:, :, 0:66]
X_valid_pose = X_valid[:, :, 0:66]
#sEMG
X_train_sEMG = X_train[:, :, 66:70]
X_valid_sEMG = X_valid[:, :, 66:70]

In [4]:
xyz_correlations = []
for i in range(66):  # 遍历每个特征
    repeated_y_train = np.repeat(y_train, X_train_pose.shape[1], axis=0)
    coef, _ = spearmanr(X_train_pose[:, :, i].flatten(), repeated_y_train[:X_train_pose[:, :, i].flatten().shape[0]])
    xyz_correlations.append(coef)

avg_xyz_correlation = np.mean(xyz_correlations)

sEMG_correlations = []
for i in range(4):  # sEMG数据有4个通道
    repeated_y_train_sEMG = np.repeat(y_train, X_train_sEMG.shape[1], axis=0)
    coef, _ = spearmanr(X_train_sEMG[:, :, i].flatten(), repeated_y_train_sEMG[:X_train_sEMG[:, :, i].flatten().shape[0]])
    sEMG_correlations.append(coef)

avg_sEMG_correlation = np.mean(sEMG_correlations)

print(avg_xyz_correlation)
print(avg_sEMG_correlation)

all_correlations = []
all_correlations.append(avg_xyz_correlation)
all_correlations.append(avg_sEMG_correlation)

abs_correlations = np.abs(all_correlations)
normalized_weights = abs_correlations / np.sum(abs_correlations)
print(normalized_weights)

0.1111111111111111
0.1111111111111111
[0.5 0.5]


In [7]:
def cnn_normal(input_shape):
    """
    Creates a simple 1D CNN model for processing time series data with a dynamic input shape.

    Args:
    input_shape (tuple): The shape of the input data, excluding the batch size.

    Returns:
    keras.engine.training.Model: A 1D CNN model.
    """
    input_data = Input(shape=input_shape)

    # Convolutional layer block 1
    x = Conv1D(32, 3, padding='same')(input_data)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    # Convolutional layer block 2
    x = Conv1D(64, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    # Convolutional layer block 3
    x = Conv1D(128, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)

    # Flattening the output and adding Dense layers
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    out = Dense(2, activation='softmax')(x)  # Assuming a binary classification task

    model = Model(inputs=input_data, outputs=out)
    return model

In [9]:
#分割成两个模态
model_pose = cnn_normal(input_shape=(180, 66))
y_pred_pose, y_true_pose, H_pose = model_pipeline(model_pose, X_train_pose, y_train, X_valid_pose, y_valid, epoch=30)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30
11/11 [==============================] - 1s 73ms/step - loss: 0.4435 - binary_accuracy: 0.6931 - val_loss: 0.7295 - val_binary_accuracy: 0.2740 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/30
11/11 [==============================] - 1s 57ms/step - loss: 0.3261 - binary_accuracy: 0.6588 - val_loss: 0.1508 - val_binary_accuracy: 0.6473 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/30
11/11 [==============================] - 1s 57ms/step - loss: 0.1808 - binary_accuracy: 0.6921 - val_loss: 0.1885 - val_binary_accuracy: 0.4054 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/30
11/11 [==============================] - 1s 56ms/step - loss: 0.1273 - binary_accuracy: 0.7995 - val_loss: 0.3293 - val_binary_accuracy: 0.0913 - lr: 0.0010

Epoch 5

In [13]:
model_sEMG = cnn_normal(input_shape=(180, 4))
y_pred_sEMG, y_true_sEMG, H_sEMG = model_pipeline(model_sEMG, X_train_sEMG, y_train, X_valid_sEMG, y_valid, epoch=30)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30
11/11 [==============================] - 1s 59ms/step - loss: 0.5327 - binary_accuracy: 0.7143 - val_loss: 0.1714 - val_binary_accuracy: 0.8613 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/30
11/11 [==============================] - 0s 45ms/step - loss: 0.1399 - binary_accuracy: 0.7518 - val_loss: 0.1466 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/30
11/11 [==============================] - 0s 45ms/step - loss: 0.1174 - binary_accuracy: 0.7937 - val_loss: 0.1301 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/30
11/11 [==============================] - 1s 46ms/step - loss: 0.1122 - binary_accuracy: 0.8031 - val_loss: 0.1269 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 5

In [14]:
weight_pose = normalized_weights[0]
weight_sEMG = normalized_weights[1]


final_pred = (y_pred_pose * weight_pose + y_pred_sEMG * weight_sEMG) / (weight_pose + weight_sEMG)
final_pred = np.round(final_pred).astype(int)


print(classification_report(y_true_pose, final_pred))
print(confusion_matrix(y_true_pose, final_pred))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2698
           1       0.43      0.07      0.12       171

    accuracy                           0.94      2869
   macro avg       0.69      0.53      0.54      2869
weighted avg       0.91      0.94      0.92      2869

[[2682   16]
 [ 159   12]]


In [15]:
weight_pose = normalized_weights[0]
weight_sEMG = normalized_weights[1]


final_pred = (y_pred_pose + y_pred_sEMG) / 2
final_pred = np.round(final_pred).astype(int)


print(classification_report(y_true_pose, final_pred))
print(confusion_matrix(y_true_pose, final_pred))

              precision    recall  f1-score   support

           0       0.94      0.99      0.97      2698
           1       0.43      0.07      0.12       171

    accuracy                           0.94      2869
   macro avg       0.69      0.53      0.54      2869
weighted avg       0.91      0.94      0.92      2869

[[2682   16]
 [ 159   12]]


In [16]:
#打包一起训练
model = cnn_normal(input_shape=(180, 70))
y_pred, y_true, H = model_pipeline(model, X_train, y_train, X_valid, y_valid, epoch=30)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30
11/11 [==============================] - 1s 72ms/step - loss: 0.6500 - binary_accuracy: 0.7021 - val_loss: 0.0741 - val_binary_accuracy: 0.9387 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/30
11/11 [==============================] - 1s 62ms/step - loss: 0.1617 - binary_accuracy: 0.6552 - val_loss: 0.1402 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/30
11/11 [==============================] - 1s 61ms/step - loss: 0.1406 - binary_accuracy: 0.7899 - val_loss: 0.1324 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/30
11/11 [==============================] - 1s 61ms/step - loss: 0.1289 - binary_accuracy: 0.7899 - val_loss: 0.1511 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 5

In [17]:
#分割成四个模态
X_train_XYZ = X_train[:, :, :66]  # XYZ坐标
X_train_sEMG = X_train[:, :, 66:70]  # sEMG数据

# 定义模态分组
trunk_indices = [0, 7, 8, 19, 20, 21]  # 身体躯干模态索引
upper_limb_indices = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]  # 上肢模态索引
lower_limb_indices = [1, 2, 3, 4, 5, 6]  # 下肢模态索引

# 初始化相关性列表
correlations_trunk = []
correlations_upper_limb = []
correlations_lower_limb = []

# 分别计算每个模态组的相关性
for group, correlations in [(trunk_indices, correlations_trunk),
                            (upper_limb_indices, correlations_upper_limb),
                            (lower_limb_indices, correlations_lower_limb)]:
    group_correlations = []
    for i in group:
        # 获取每个XYZ坐标的展平后的数组
        X_flat = X_train[:, :, i].flatten()
        Y_flat = X_train[:, :, i+22].flatten()
        Z_flat = X_train[:, :, i+44].flatten()
        # 为了匹配X_flat, Y_flat, Z_flat的长度，我们需要正确地重复y_train
        y_repeated = np.repeat(y_train, X_train.shape[1])
        # 计算相关性
        coef_X, _ = spearmanr(X_flat, y_repeated[:len(X_flat)])
        coef_Y, _ = spearmanr(Y_flat, y_repeated[:len(Y_flat)])
        coef_Z, _ = spearmanr(Z_flat, y_repeated[:len(Z_flat)])
        # 计算平均相关系数
        avg_coef = np.mean([coef_X, coef_Y, coef_Z])
        group_correlations.append(avg_coef)
    # 计算并保存该模态组的平均相关性
    avg_group_correlation = np.mean(group_correlations)
    correlations.append(avg_group_correlation)

# 对sEMG数据计算相关性
correlations_sEMG = []
for i in range(4):
    sEMG_flat = X_train_sEMG[:, :, i].flatten()
    y_repeated_sEMG = np.repeat(y_train, X_train_sEMG.shape[1])
    coef_sEMG, _ = spearmanr(sEMG_flat, y_repeated_sEMG[:len(sEMG_flat)])
    correlations_sEMG.append(coef_sEMG)

correlation_sEMG = np.mean(correlations_sEMG)

all_correlations = []
all_correlations.append(correlations_trunk[0])

all_correlations.append(correlations_upper_limb[0])
all_correlations.append(correlations_lower_limb[0])
all_correlations.append(correlation_sEMG)


# 打印相关性结果
print("Correlations for Trunk:", correlations_trunk)
print("Correlations for Upper Limb:", correlations_upper_limb)
print("Correlations for Lower Limb:", correlations_lower_limb)
print("Correlations for sEMG signals:", correlation_sEMG)
print("Correlations for All:", all_correlations)

Correlations for Trunk: [-0.0005935154175248134]
Correlations for Upper Limb: [-5.168494671029889e-05]
Correlations for Lower Limb: [-0.000717485724721992]
Correlations for sEMG signals: -0.0029697894500262632
Correlations for All: [-0.0005935154175248134, -5.168494671029889e-05, -0.000717485724721992, -0.0029697894500262632]


In [18]:
# Calculate absolute values and normalize to get initial weights
abs_correlations = np.abs(all_correlations)
normalized_weights = abs_correlations / np.sum(abs_correlations)

# Clip weights exceeding the threshold and redistribute if necessary
max_threshold = 0.5
clipped_weights = np.clip(normalized_weights, None, max_threshold)

# Redistribute weights if any were clipped to the threshold
if np.any(clipped_weights == max_threshold):
    # Calculate the total weight to be redistributed among non-clipped weights
    total_redistribute_weight = 1 - np.sum(clipped_weights == max_threshold) * max_threshold
    # Calculate the sum of weights that are less than the threshold (these will be redistributed)
    sum_weights_to_redistribute = np.sum(clipped_weights[clipped_weights < max_threshold])
    # Adjust weights that are below the threshold
    for i, weight in enumerate(clipped_weights):
        if weight < max_threshold:
            clipped_weights[i] = weight / sum_weights_to_redistribute * total_redistribute_weight

print(clipped_weights)

[0.21777408 0.01896436 0.26326156 0.5       ]


In [22]:
# Modality groups definition
trunk_indices = [0, 7, 8, 19, 20, 21]
upper_limb_indices = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
lower_limb_indices = [1, 2, 3, 4, 5, 6]
sEMG_indices = list(range(66, 70))

# Function to prepare modality-specific datasets
def prepare_modality_data(X, indices, is_sEMG=False):
    if is_sEMG:
        return X[:, :, indices]
    else:
        all_indices = []
        for i in indices:
            all_indices.extend([i, i+22, i+44])
        return X[:, :, all_indices]

# Define modalities
modalities = {
    "Trunk": trunk_indices,
    "Upper Limb": upper_limb_indices,
    "Lower Limb": lower_limb_indices,
    "sEMG": sEMG_indices
}

predictions = {}

for modality_name, indices in modalities.items():
    is_sEMG = (modality_name == "sEMG")
    X_train_modality = prepare_modality_data(X_train, indices, is_sEMG)
    X_valid_modality = prepare_modality_data(X_valid, indices, is_sEMG)

    input_shape = (X_train_modality.shape[1], X_train_modality.shape[2])
    model = cnn_normal(input_shape)  # Use cnn_normal instead of stacked_lstm
    y_pred_modality, y_true_modality, _ = model_pipeline(model, X_train_modality, y_train, X_valid_modality, y_valid)

    predictions[modality_name] = y_pred_modality



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 1s 55ms/step - loss: 0.6954 - binary_accuracy: 0.6664 - val_loss: 0.1532 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 49ms/step - loss: 0.1435 - binary_accuracy: 0.7877 - val_loss: 0.1461 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 48ms/step - loss: 0.1385 - binary_accuracy: 0.7911 - val_loss: 0.1700 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 48ms/step - loss: 0.1226 - binary_accuracy: 0.7927 - val_loss: 0.1475 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 5


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 1s 59ms/step - loss: 0.3910 - binary_accuracy: 0.6608 - val_loss: 0.1194 - val_binary_accuracy: 0.9111 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 52ms/step - loss: 0.2895 - binary_accuracy: 0.6995 - val_loss: 0.1832 - val_binary_accuracy: 0.3775 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 51ms/step - loss: 0.1374 - binary_accuracy: 0.8095 - val_loss: 0.1232 - val_binary_accuracy: 0.7783 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 51ms/step - loss: 0.1132 - binary_accuracy: 0.8230 - val_loss: 0.2473 - val_binary_accuracy: 0.1199 - lr: 0.0010

Epoch 5


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 1s 57ms/step - loss: 0.3821 - binary_accuracy: 0.6367 - val_loss: 0.2199 - val_binary_accuracy: 0.7337 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 50ms/step - loss: 0.2784 - binary_accuracy: 0.5954 - val_loss: 0.1429 - val_binary_accuracy: 0.6720 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 48ms/step - loss: 0.1376 - binary_accuracy: 0.7370 - val_loss: 0.1047 - val_binary_accuracy: 0.9129 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 49ms/step - loss: 0.1293 - binary_accuracy: 0.7901 - val_loss: 0.1243 - val_binary_accuracy: 0.9278 - lr: 0.0010

Epoch 5


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 1s 58ms/step - loss: 0.3411 - binary_accuracy: 0.6782 - val_loss: 0.1775 - val_binary_accuracy: 0.0596 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 51ms/step - loss: 0.1526 - binary_accuracy: 0.7378 - val_loss: 0.1595 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 49ms/step - loss: 0.1166 - binary_accuracy: 0.8071 - val_loss: 0.1515 - val_binary_accuracy: 0.9397 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 50ms/step - loss: 0.1146 - binary_accuracy: 0.8136 - val_loss: 0.1411 - val_binary_accuracy: 0.9397 - lr: 0.0010

Epoch 5

In [23]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

predictions_list = [predictions[modality] for modality in modalities.keys()]

weighted_predictions = np.zeros(predictions_list[0].shape)

# Apply the clipped and redistributed weights to the predictions
for i, prediction in enumerate(predictions_list):
    # print(clipped_weights[i])
    weighted_predictions += prediction * clipped_weights[i]
    # weighted_predictions += prediction / 4

# print(weighted_predictions)


final_predictions = np.round(weighted_predictions)

# Convert y_valid to class indices if it's in one-hot encoding
y_valid_indices = np.argmax(y_valid, axis=1)

# Evaluate the combined predictions
accuracy = accuracy_score(y_valid_indices, final_predictions)
classification_report_result = classification_report(y_valid_indices, final_predictions)
confusion_matrix_result = confusion_matrix(y_valid_indices, final_predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report_result)
print("Confusion Matrix:\n", confusion_matrix_result)

Accuracy: 0.9320320669222726
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.97      0.96      2698
           1       0.42      0.36      0.38       171

    accuracy                           0.93      2869
   macro avg       0.69      0.66      0.67      2869
weighted avg       0.93      0.93      0.93      2869

Confusion Matrix:
 [[2613   85]
 [ 110   61]]


In [25]:
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, LSTM, Activation, BatchNormalization, Multiply, GlobalAveragePooling1D, Permute, Reshape, concatenate
from keras.models import Model
import keras.backend as K

def enhanced_cnn_with_attention(input_shape):
    """
    Creates an enhanced 1D CNN model with attention mechanism and additional features for processing time series data.

    Args:
    input_shape (tuple): The shape of the input data, excluding the batch size.

    Returns:
    keras.engine.training.Model: An enhanced 1D CNN model with attention.
    """
    input_data = Input(shape=input_shape)

    # Convolutional layer block 1
    x = Conv1D(32, 5, padding='same')(input_data)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(32, 5, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.25)(x)

    # Convolutional layer block 2
    x = Conv1D(64, 5, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(64, 5, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.25)(x)

    # Convolutional layer block 3
    x = Conv1D(128, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv1D(128, 3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPooling1D(2)(x)
    x = Dropout(0.25)(x)

    # Attention Mechanism
    attention_probs = Dense(128, activation='softmax', name='attention_vec')(x)
    attention_mul = Multiply()([x, attention_probs])

    # Global Average Pooling
    x = GlobalAveragePooling1D()(attention_mul)

    # Additional Dense Layers
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)

    out = Dense(2, activation='softmax')(x)  # Assuming a binary classification task

    model = Model(inputs=input_data, outputs=out)
    return model


In [26]:
#分割成两个模态
model_pose = enhanced_cnn_with_attention(input_shape=(180, 66))
y_pred_pose, y_true_pose, H_pose = model_pipeline(model_pose, X_train_pose, y_train, X_valid_pose, y_valid, epoch=30)

model_sEMG = enhanced_cnn_with_attention(input_shape=(180, 4))
y_pred_sEMG, y_true_sEMG, H_sEMG = model_pipeline(model_sEMG, X_train_sEMG, y_train, X_valid_sEMG, y_valid, epoch=30)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30
11/11 [==============================] - 2s 123ms/step - loss: 0.1630 - binary_accuracy: 0.7382 - val_loss: 0.1035 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/30
11/11 [==============================] - 1s 107ms/step - loss: 0.1350 - binary_accuracy: 0.7899 - val_loss: 0.1001 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/30
11/11 [==============================] - 1s 107ms/step - loss: 0.1295 - binary_accuracy: 0.7899 - val_loss: 0.1020 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/30
11/11 [==============================] - 1s 107ms/step - loss: 0.1271 - binary_accuracy: 0.7899 - val_loss: 0.1019 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epo


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30
11/11 [==============================] - 2s 104ms/step - loss: 0.1616 - binary_accuracy: 0.7380 - val_loss: 0.1296 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/30
11/11 [==============================] - 1s 93ms/step - loss: 0.1337 - binary_accuracy: 0.7899 - val_loss: 0.1053 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/30
11/11 [==============================] - 1s 94ms/step - loss: 0.1209 - binary_accuracy: 0.7899 - val_loss: 0.1109 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/30
11/11 [==============================] - 1s 96ms/step - loss: 0.1180 - binary_accuracy: 0.7899 - val_loss: 0.1173 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 

In [27]:
weight_pose = normalized_weights[0]
weight_sEMG = normalized_weights[1]


final_pred = (y_pred_pose * weight_pose + y_pred_sEMG * weight_sEMG) / (weight_pose + weight_sEMG)
final_pred = np.round(final_pred).astype(int)


print(classification_report(y_true_pose, final_pred))
print(confusion_matrix(y_true_pose, final_pred))

              precision    recall  f1-score   support

           0       0.96      0.86      0.91      2698
           1       0.18      0.49      0.26       171

    accuracy                           0.83      2869
   macro avg       0.57      0.67      0.58      2869
weighted avg       0.92      0.83      0.87      2869

[[2309  389]
 [  87   84]]


In [28]:
#打包一起训练
model = enhanced_cnn_with_attention(input_shape=(180, 70))
y_pred, y_true, H = model_pipeline(model, X_train, y_train, X_valid, y_valid, epoch=30)


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30
11/11 [==============================] - 2s 137ms/step - loss: 0.1637 - binary_accuracy: 0.7392 - val_loss: 0.0933 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/30
11/11 [==============================] - 1s 125ms/step - loss: 0.1360 - binary_accuracy: 0.7899 - val_loss: 0.0939 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/30
11/11 [==============================] - 1s 125ms/step - loss: 0.1284 - binary_accuracy: 0.7899 - val_loss: 0.1018 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/30
11/11 [==============================] - 1s 124ms/step - loss: 0.1255 - binary_accuracy: 0.7899 - val_loss: 0.1050 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epo

In [29]:
# Modality groups definition
trunk_indices = [0, 7, 8, 19, 20, 21]
upper_limb_indices = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
lower_limb_indices = [1, 2, 3, 4, 5, 6]
sEMG_indices = list(range(66, 70))

# Function to prepare modality-specific datasets
def prepare_modality_data(X, indices, is_sEMG=False):
    if is_sEMG:
        return X[:, :, indices]
    else:
        all_indices = []
        for i in indices:
            all_indices.extend([i, i+22, i+44])
        return X[:, :, all_indices]

# Define modalities
modalities = {
    "Trunk": trunk_indices,
    "Upper Limb": upper_limb_indices,
    "Lower Limb": lower_limb_indices,
    "sEMG": sEMG_indices
}

predictions = {}

for modality_name, indices in modalities.items():
    is_sEMG = (modality_name == "sEMG")
    X_train_modality = prepare_modality_data(X_train, indices, is_sEMG)
    X_valid_modality = prepare_modality_data(X_valid, indices, is_sEMG)

    input_shape = (X_train_modality.shape[1], X_train_modality.shape[2])
    model = enhanced_cnn_with_attention(input_shape)  # Use cnn_normal instead of stacked_lstm
    y_pred_modality, y_true_modality, _ = model_pipeline(model, X_train_modality, y_train, X_valid_modality, y_valid)

    predictions[modality_name] = y_pred_modality



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 2s 107ms/step - loss: 0.1629 - binary_accuracy: 0.7257 - val_loss: 0.1214 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 97ms/step - loss: 0.1369 - binary_accuracy: 0.7899 - val_loss: 0.0975 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 97ms/step - loss: 0.1276 - binary_accuracy: 0.7899 - val_loss: 0.1018 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 99ms/step - loss: 0.1255 - binary_accuracy: 0.7899 - val_loss: 0.1014 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 2s 115ms/step - loss: 0.1629 - binary_accuracy: 0.7688 - val_loss: 0.0972 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 106ms/step - loss: 0.1365 - binary_accuracy: 0.7899 - val_loss: 0.0922 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 104ms/step - loss: 0.1297 - binary_accuracy: 0.7899 - val_loss: 0.0968 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 110ms/step - loss: 0.1273 - binary_accuracy: 0.7899 - val_loss: 0.0991 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epo


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 2s 111ms/step - loss: 0.1662 - binary_accuracy: 0.7450 - val_loss: 0.1086 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 102ms/step - loss: 0.1352 - binary_accuracy: 0.7899 - val_loss: 0.0963 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 102ms/step - loss: 0.1289 - binary_accuracy: 0.7899 - val_loss: 0.1039 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 102ms/step - loss: 0.1266 - binary_accuracy: 0.7899 - val_loss: 0.1066 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epo


Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/50
11/11 [==============================] - 2s 111ms/step - loss: 0.1574 - binary_accuracy: 0.7873 - val_loss: 0.1232 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/50
11/11 [==============================] - 1s 102ms/step - loss: 0.1283 - binary_accuracy: 0.7899 - val_loss: 0.1114 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/50
11/11 [==============================] - 1s 101ms/step - loss: 0.1193 - binary_accuracy: 0.7899 - val_loss: 0.1178 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/50
11/11 [==============================] - 1s 102ms/step - loss: 0.1157 - binary_accuracy: 0.7899 - val_loss: 0.1222 - val_binary_accuracy: 0.9404 - lr: 0.0010

Epo

In [30]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

predictions_list = [predictions[modality] for modality in modalities.keys()]

weighted_predictions = np.zeros(predictions_list[0].shape)

# Apply the clipped and redistributed weights to the predictions
for i, prediction in enumerate(predictions_list):
    # print(clipped_weights[i])
    weighted_predictions += prediction * clipped_weights[i]
    # weighted_predictions += prediction / 4

# print(weighted_predictions)


final_predictions = np.round(weighted_predictions)

# Convert y_valid to class indices if it's in one-hot encoding
y_valid_indices = np.argmax(y_valid, axis=1)

# Evaluate the combined predictions
accuracy = accuracy_score(y_valid_indices, final_predictions)
classification_report_result = classification_report(y_valid_indices, final_predictions)
confusion_matrix_result = confusion_matrix(y_valid_indices, final_predictions)

print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report_result)
print("Confusion Matrix:\n", confusion_matrix_result)

Accuracy: 0.9194841408156152
Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.94      0.96      2698
           1       0.38      0.54      0.44       171

    accuracy                           0.92      2869
   macro avg       0.67      0.74      0.70      2869
weighted avg       0.93      0.92      0.93      2869

Confusion Matrix:
 [[2546  152]
 [  79   92]]
